<a href="https://colab.research.google.com/github/pratikg47/Camouflaged-target-acquisition-using-YOLOv8/blob/main/Camouflage_with_Alert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import matplotlib.pyplot as plt
from ultralytics import YOLO
from google.colab import files

In [ ]:
# Function to send alert emails
def send_alert(subject, body, to_email):
    from_email = "blacksmith8983@gmail.com"
    password = "ikukjwgsqylejjut"

    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com: 587')
        server.starttls()
        server.login(from_email, password)
        text = msg.as_string()
        server.sendmail(from_email, to_email, text)
        server.quit()
        print("Alert sent successfully!")
    except Exception as e:
        print(f"Failed to send alert: {e}")


In [ ]:
# Perform YOLOv8 object detection and count persons
def detect_persons_yolo(model, frame):
    results = model(frame)
    person_count = 0
    for result in results:
        # Iterate over each detected object
        for bbox in result.boxes:
            if bbox.cls == 0:  # YOLOv8 class ID for 'person' is 0
                person_count += 1
    return person_count

In [ ]:
# Camouflage detection function (example logic)
def detect_camouflage(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale frame
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection using Canny
    edges = cv2.Canny(blurred, 50, 150)

    # Perform contour detection
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if any contour is found
    if contours:
        print("Camouflage detected!")
        return "camouflage_detected"
    else:
        print("No camouflage detected.")
        return "no_camouflage"


In [ ]:
# Upload the video file and YOLOv8 model to Colab
#uploaded = files.upload()

# Define paths to the uploaded files
video_path = '/content/2 sec sample video.mp4'  # Replace with your actual video file name
model = YOLO("yolov8l-seg.pt")   # segmentation model

# Open the video file
cap = cv2.VideoCapture(video_path)

100%|██████████| 88.1M/88.1M [00:00<00:00, 241MB/s]


In [ ]:
if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detection_result = detect_camouflage(frame)

        if detection_result == "camouflage_detected":
            person_count = detect_persons_yolo(model, frame)

            # Generate alert if camouflage is detected
            subject = "Camouflage Detection Alert"
            body = f"A camouflage object has been detected in the video. Number of persons detected: {person_count}."
            to_email = "amanmaner011@gmail.com"
            send_alert(subject, body, to_email)
            #break  # Alert once and stop processing further frames

        # # Display the current frame (optional)
        # cv2.imshow('Frame', frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #   break
    # Release the video capture object and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()


Camouflage detected!

0: 384x640 9 persons, 1 backpack, 2 kites, 2663.8ms
Speed: 16.0ms preprocess, 2663.8ms inference, 1678.3ms postprocess per image at shape (1, 3, 384, 640)
Alert sent successfully!
